In [1]:
mixtral_run_name = ["mistralai/mixtral-8x7b-instruct-v0.1", "mistralai/mistral-7b-instruct-v0.2"]
Llama_run_name = ["meta/llama-2-70b:14ce4448d5e7e9ed0c37745ac46eca157aab09061f0c179ac2b323b5de56552b", "meta/llama-2-70b-chat:2d19859030ff705a87c746f7e96eea03aefb71f166725aee39692f1476566d48"]

In [2]:
import sys
import os
import glob
from pathlib import Path

# Assuming src is in the parent directory of the current working directory
sys.path.append(os.path.join(Path.cwd().parent, 'src' ) ) 
sys.path.append(os.path.join(Path.cwd().parent, 'src' , 'utils') ) 

from utils import load_data, json_arr_to_file,  run_hf_api_call

from utils import preprocess_options_and_labels, intention_prompt_second_hf, intention_prompt_first_hf


In [7]:
run_name = mixtral_run_name[0]
os.makedirs(f'./data/{run_name}', exist_ok=True)

for f in ['helpful', 'harmless']:
    os.makedirs(f'../data/{run_name}/processed/{f}' , exist_ok=True)

    topics_file = glob.glob(f'../data/old_tests/intention-gpt-4/{f}/*')
    for file in topics_file:

        print(file)

        file_name = file.split('/')[-1]
        hh = file.split('/')[-2]
        full_json = load_data(file)
        for item in full_json:
            op = item['options']
            lab  = item['labels']
            scenario = item['scenario']
            adapt_outcome = item['adapt_outcome']

            # permutate options and save mapping
            mapping, pr_string = preprocess_options_and_labels(op, lab)
            item['mapping_given_to_model'] = mapping

            # load prompts
            first_prompt = intention_prompt_first_hf(scenario, pr_string)
            second_prompt = intention_prompt_second_hf(scenario, pr_string, adapt_outcome)
            first_response = run_hf_api_call(first_prompt, run_name)
            item['first response'] = first_response
            second_response = run_hf_api_call(second_prompt, run_name)
            item['second response'] = second_response

        write_path = os.path.join(f'../data/{run_name}/processed/', hh, file_name)
        json_arr_to_file(full_json, write_path, indent=4)


../data/old_tests/intention-gpt-4/helpful/10--Travel.json
Calling API with mistralai/mixtral-8x7b-instruct-v0.1


ModelError: 'Request 0 already exists.'

In [5]:
print(second_response)

3
